In [9]:
# Blue Rectangle Eyes Detection
import cv2

# Load the cascade classifier for eyes
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Start the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect eyes in the grayscale frame
    eyes = eye_cascade.detectMultiScale(gray, 1.3, 5)

    # Draw a rectangle around the eyes
    for (x, y, w, h) in eyes:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    cv2.imshow("Webcam", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [14]:
import cv2
import dlib
import numpy as np
import time

def pupil_detection(frame,predictor,detector):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    for rect in rects:
        shape = predictor(gray, rect)
        shape = np.array([[p.x, p.y] for p in shape.parts()])

        # extract left and right eye coordinates
        left_eye = shape[36:42]
        right_eye = shape[42:48]

        # find center of the left and right eye
        left_eye_center = left_eye.mean(axis=0).astype("int")
        right_eye_center = right_eye.mean(axis=0).astype("int")

        # Draw the dots
        cv2.circle(frame, tuple(left_eye_center), 3, (0, 255, 0), -1)
        cv2.circle(frame, tuple(right_eye_center), 3, (0, 255, 0), -1)

    return frame

# Load the face detection model
detector = dlib.get_frontal_face_detector()
# Load the facial landmark predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
# Start webcam
cap = cv2.VideoCapture(0)

start_time = time.time()
frame_count = 0
fps = 0

while True:
    _, frame = cap.read()
    frame = pupil_detection(frame,predictor,detector)
    frame_count += 1
    
    if time.time() - start_time > 1:
        fps = frame_count / (time.time() - start_time)
        frame_count = 0
        start_time = time.time()

    # Camera Horizontal Inversion
    frame = cv2.flip(frame, 1)
    
    cv2.putText(frame, "FPS: {:.2f}".format(fps), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    

    cv2.imshow("Webcam", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [12]:
# Versao trackeador pupilas com linha vermelha no meio
import cv2
import dlib
import numpy as np

# Load pre-trained face landmark detector
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    # Read frame from webcam
    ret, frame = cap.read()

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = detector(gray, 0)

    # Loop over faces
    for face in faces:
        # Get face landmarks
        landmarks = predictor(gray, face)
        landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])

        # Extract left and right eye landmarks
        left_eye = landmarks[36:42]
        right_eye = landmarks[42:48]

        # Calculate middle points of left and right eye
        left_eye_center = left_eye.mean(axis=0).astype("int")
        right_eye_center = right_eye.mean(axis=0).astype("int")

        # Draw circles at eye center points
        cv2.circle(frame, tuple(left_eye_center), 3, (0, 0, 255), -1)
        cv2.circle(frame, tuple(right_eye_center), 3, (0, 0, 255), -1)

        # Draw line connecting the eye centers
        cv2.line(frame, tuple(left_eye_center), tuple(right_eye_center), (0, 0, 255), 2)

    # Display the frame
    cv2.imshow("Gaze Direction", frame)

    # Break loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release webcam and close all windows
cap.release()
cv2.destroyAllWindows()